<a href="https://colab.research.google.com/github/libra3910/diveintocode-ml/blob/master/FasterR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 論文読解

(1) 物体検出の分野にはどういった手法が存在したか。

■類似した特徴をもつ画素をグルーピングする手法

```py
①SelectiveSearch
　参照「https://blog.shikoan.com/selective-search-rcnn/」
　以下の段階で領域提案を実施する。古典的な教師無し学習ライブラリ。
　・Felzenszwalb法による画像のセグメンテーション　→　似たような領域をセグメント化する。サイズはパラメータ指定。
  ・ヒストグラムベースの特徴量を作る　→　セグメントごとに色ヒストグラム（R、G、B）と領域を示すインデックスを付与。
  ・テクスチャの勾配を取る　→　LBP（Local Binary Pattern）。ex：RGBをグレースケール化する。
  ・ヒストグラムを取る
  　・色のヒストグラム（チャンネルごとにヒストグラムを取る）
  　・テクスチャ勾配のヒストグラム（RGB画像の各チャンネル単位のLBPについて、ヒストグラムを取る）
  ・Neighborの作成　→　重なっているRegionのペアを列挙。
  ・全てのNeighborに対し、Region同士の類似度を計算
  ・Hierarchal Search　→　類似度が高いもの同士を束ねる
```

```py
②CPMC
　前景として配置された点(Seeds)の周囲にグラフカット処理が行われ、それらを基に物体領域を認識するという方法。処理速度が非常に遅いが
　かなりの高精度で物体領域が検出できるようです。また強みとしてObjectnessでは得られない物体の輪郭が高精度で得られるという点があり、
　目的の物体を背景から切り離して処理したい時などに適している。
```

■スライディングウインドウズ

```py
①EdgeBoxes
　エッジから直接オブジェクトのバウンディングボックスを生成するアプローチ。
　エッジは、セグメントと同様に、画像を単純化して表現
```

■ディープネットワーク

```py
①R-CNN
　主に分類器として機能し、オブジェクトの境界予測は行わない。
②OverFeat法
　完全連結層を学習して，物体の境界線を予測するように学習する。
③MultiBox
　1枚の画像の切り抜きや、単一の画像クロップまたは複数の大きな画像クロップ（例えば、224×224）に適用される。
```

(2) Fasterとあるが、どういった仕組みで高速化したのか。

```py
Faster R-CNNは、領域提案と物体検出を分離したツーステージ検出モデルで行うRPNの採用と、アンカーおよびアンカーボックスをベースとした領域提案方式により提案する領域数を最小化する方法採用し、画像処理速度を向上させた。
ベンチマークテストでは、領域提案数2000のSS（Selective Search）+Fast R-CNNよりも、領域提案数300のRPN+FAST R-CNNのほうが、処理速度が早かった。
```

(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

```py
One-Stage手法は領域提案と物体検出を一括で処理を行う方式で、OverHeat法がこれを採用していることを紹介している。それに対し、領域提案と
物体検出を分離した方式がTwo-Stage手法で、Faster R-CNNがこの方式を採用している。
PASCAL VOC 2007 test setを使用したベンチマークテストでは、Two-Stage手法のほうが、One-Stage手法よりも平均適合率（mAP）が向上した。
```

(4) RPNとは何か。

```py
参照：Faster R-CNNにおけるRPNの世界一分かりやすい解説
FasterR-CNNの機能の一つである、物体か背景かを識別する機能を担うモデルである。古典的手法である「SelectedSearch」に代わって
実装されている。主な詳細機能は以下の通り。
　①VGG16モデル等による畳み込み、FeatureMapを作成。
　②FeatureMapに対してAnchorを設定。
　③Anchorごとに9種類のAnchorBoxを作成。
　④GroundTruth（アノテーション作業を通じて人間が用意する特徴画像情報）とAnchorBoxを比較する。
　⑤ズレ具合や物体か背景かの判別を実施し、clsクラス（物体か背景か）とregクラス（4通りのズレ具合）を作成する。
以降は、RoIプーリングおよび全結合層に引き渡され、物体が何者であるかが判別される。
```

(5) RoIプーリングとは何か。

```py
様々な画像サイズからVGG16モデルを通して作成されたFeatureMapのサイズを固定するためのプーリング層。これにより画像サイズに影響されず、
学習と予測が可能になる。
```

(6) Anchorのサイズはどうするのが適切か。

```py
Anchor Boxのサイジングは、PASCAL VOC 2007 test setを使用したベンチマークテストでは、スケールでは３セッティング、アスペクト比も3セッティング（合計9セッティング）が最も平均適合率が良かった。ただし、３スケール、１アスペクト比でもほぼ一緒であったため、このサイジングでも結果は差が少ないと思われる。
```

(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

```py
PASCAL VOC 2007 test set および 2012 test setを使用したベンチマークテストでは、先行研究であるSS（Selective Search）の2000の領域提案
に比べ、RPN＋VGG（shared）の300領域提案のほうがmAP（平均適応率）が良い指標値が得られた。また、K40GPUにおける処理時間の速さを比べる
ベンチマークテストでは、SS+Fast R-CNNよりもRPN+Fast R-CNNのほうが圧倒的に処理速度が速かった。
```

(8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

```py
引用元①：ＦＰＮ発明の概要
引用元②：Feature Pyramid Networks for Object Detection
従来の物体検出モデルとして、領域提案および物体検出を一体で行うワンステージ検出モデルを採用していた。そこでFaster R-CNNは、
領域提案と物体検出を分離したツーステージ検出モデルで行うRPNと、アンカーをベースとした領域提案方式を採用し、画像処理速度を向上させた。
しかし、サイズが大きい画像に対しては検出精度が良好であるが、サイズが小さい画像に対しては、検出精度があまり良くないという問題点が
あった。そこでTsung-Yi Lin氏らがFPN（FeaturePyramidNetwork）を提唱し、ROIプーリングの部分をRPNからFPNに置き換えることにより、
小サイズの検出精度を向上させることができた。
```